# KNN-1: train with partial curves 

My idea is to used one of the partial cycle files to train the KNN model. For that, I need to get the distances between each of the partial curves with all the full curves. So for each partial curve, choose the closer full curve.
The result is an array with all the distances for each partial cycle. The number of cycle is the label of each distance.
With that i train the model.

Then, the user will give us a curve. We need to get first the distance of this curve to all our full curves and get the closer one.
That distance is what i feed the model to make a prediction of the cycle number.

We can used one file of partial curves as train set and then other as test set.

**I tried to do this, but the code for getting the distance for each partial curve takes FOREVER!. So i only used 2 partial curves to train. Of course this is not okey, but i just wanted to see if this made any sense**

In [1]:
import pandas as pd
import numpy as np
import import_data
import sort_data
import matplotlib.pyplot as plt

%matplotlib inline

/Users/rossana13/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Load the data. We need full curves and one partial curve (that represents the user's curve)

PL03c,PL03d = sort_data.charge_discharge('converted_PL03.mat')
full_curvesc,full_curvesd = sort_data.charge_discharge('converted_PL11.mat')

PL10c,PL10d = sort_data.charge_discharge('data/converted_PL10.mat')

In [3]:
a = PL03c[444]['voltage'].values.flatten()
b = full_curvesc[2][['voltage']].values.flatten()

In [4]:
print(len(a),len(b))

802 1209


In [5]:
#Make a shorter partial curve dictionary to uses as train set (because if not the code takes too long)
PL03c_short = {}
for idx in range(1, len(PL03d.keys()), 10):
        if idx in PL03d.keys():
            if idx not in PL03c_short.keys():
                PL03c_short[idx] = PL03c[idx]
PL03c_short.keys()        

dict_keys([1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 201, 211, 221, 231, 241, 251, 261, 271, 281, 291, 301, 311, 321, 331, 351, 361, 371, 381, 391, 401, 411, 421, 431, 441, 451, 461, 471, 481, 491, 501, 511, 521, 531, 541, 551, 561, 571, 581, 591, 601, 611, 641, 651, 661, 671, 681, 691, 701, 811, 821, 831, 841, 851, 861, 871, 881, 891, 901, 911, 921, 931, 941, 951, 961, 971, 981, 991, 1001, 1011, 1021, 1031, 1041, 1051, 1061])

In [6]:
#Make a shorter partial curve dictionary to uses as train set (because if not the code takes too long)
full_curvesc_short = {}
for idx in range(1, len(full_curvesc.keys()), 10):
        if idx in full_curvesc.keys():
            if idx not in full_curvesc_short.keys():
                full_curvesc_short[idx] = full_curvesc[idx]
full_curvesc_short.keys()

dict_keys([1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 201, 211, 221, 231, 241, 251, 261, 271, 281, 291, 301, 311, 321, 331, 341, 351, 361, 371, 381, 391, 401, 481, 491, 501, 511, 521, 531, 541, 551, 561, 571, 581, 591, 601, 611, 621, 631, 641, 651, 661, 671, 681, 691, 701, 711, 721, 731, 741, 751, 761, 771, 781, 791, 801, 811, 821, 831])

In [7]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

def curve_distance(a,b):
#Inputs are two nparray    
    distance,path = fastdtw(a, b, dist=euclidean)
    return distance

In [8]:
def distance_cycle_to_full(cycle,dic1,dic2):
#Inputs are 2 dictionaries and the number of cycle that is being tested
    if cycle not in dic1.keys():
        pass
    else:
        
        a = dic1[cycle][['voltage']].values.flatten()
        Y = []
    
        for idx in range(1,len(dic2.keys()) + 1):
            if idx in dic2.keys():
                b = dic2[idx][['voltage']].values.flatten()
                if len(b) > 100: #This is to mae sure the full curve has values (check curve 102, it has only 1 value) 
                    b = dic2[idx][['voltage']].values.flatten() 
                    d = curve_distance(a,b)
            
                    A = [idx,d]
                    Y.append(A)
                else:
                    pass
            else:
                pass
    df = pd.DataFrame(Y, columns = ['Cycle','Distance'])
    df = df.sort_values(by = ['Distance'])
    
    Distance = df['Distance'][df.index[0]]
    return Distance

In [9]:
def partial_to_full(dic1,dic2):
    C = []
    D = []
    
    for i in range(1, 1000):
        if i in dic1.keys():
            Distance = distance_cycle_to_full(i,dic1,dic2)
            C.append(i)
            D.append(Distance)
        else:
            pass
    #For the model, we need the lists as NumPy arrays
    D = np.array(D)
    C = np.array(C)
    return D,C

In [10]:
len(PL03c_short.keys())

94

In [15]:
distance_cycle_to_full(1,PL03c_short,full_curvesc_short)

182.61686372756958

In [19]:
distance_cycle_to_full(11,PL03c_short,full_curvesc_short)

114.33854365348816

In [11]:
D, C = partial_to_full(PL03c_short,full_curvesc_short)

Train the model:

In [20]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(D.reshape(-1, 1), C) #Train the classifier with the distance and the corresponding cycles

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [21]:
#For predicting first we need the distance between the testing curve to the full curves
def distance_to_full(curve,dic):
 
    #Convert the data in the data frame to a NumPy array
    a = curve.values.flatten() 
    Y = []
    
    #Calculate the distances between the testing curve and each of the full curve cycle
    for i in range(1,len(dic.keys()) + 1):
        if i in dic.keys():
            b = dic[i][['voltage']].values.flatten()
            if len(b) > 100: #This is to mae sure the full curve has values (check curve 102, it has only 1 value)
                d = curve_distance(a,b)
            
                A = [i,d]
                Y.append(A)
            else:
                pass
        else:
            pass
    #Creat a Pandas dataframe with the distances for each cycle. Sort the data frame by distance
    df = pd.DataFrame(Y, columns = ['Cycle','Distance'])
    df = df.sort_values(by = ['Distance'])
    return df

In [22]:
df = distance_to_full(PL10c[700]['voltage'],full_curvesc_short)

In [23]:
Dist = df['Distance'][df.index[0]]
Dist

92.97753834724426

In [24]:
neigh.predict(Dist)

array([41])

In [25]:
df = distance_to_full(PL10c[44]['voltage'],full_curvesc_short)

In [26]:
Dist = df['Distance'][df.index[0]]
Dist

121.85904240608215

In [27]:
neigh.predict(Dist)

array([181])

In [29]:
df = distance_to_full(PL10c[12]['voltage'],full_curvesc_short)

In [30]:
Dist = df['Distance'][df.index[0]]
Dist

118.1771981716156

In [31]:
neigh.predict(Dist)

array([231])